# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
df = pd.read_csv("Orders.csv")
df.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [7]:
# Sub Problem 1: How to aggregate the amount_spent for unique customers?
customers = df.groupby(['CustomerID','Country']).agg({'amount_spent':sum})
customers

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40
...,...,...
18280,United Kingdom,180.60
18281,United Kingdom,80.82
18282,United Kingdom,178.05


In [19]:
# Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
quantiles = customers['amount_spent'].quantile([0.75, 0.95])
quantiles

0.75    1661.195
0.95    5774.278
Name: amount_spent, dtype: float64

In [20]:
labels=['Normal','Preferred','VIP']
customers['customer_type'] = pd.cut(customers['amount_spent'], bins=[-np.inf, quantiles[0.75], quantiles[0.95], np.inf], labels=labels)
customers

,,amount_spent,customer_type
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP
12347,Iceland,4310.00,Preferred
12348,Finland,1797.24,Preferred
12349,Italy,1757.55,Preferred
12350,Norway,334.40,Normal
...,...,...,...
18280,United Kingdom,180.60,Normal
18281,United Kingdom,80.82,Normal
18282,United Kingdom,178.05,Normal


In [21]:
# Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?
vip_customer = customers[customers['customer_type']=='VIP']
preferred_customer = customers[customers['customer_type']=='Preferred']

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [22]:
vip_country = customers.groupby(['Country']).agg({'amount_spent':sum}).sort_values('amount_spent', ascending=False)
vip_country

,amount_spent
Country,
United Kingdom,7308391.554
Netherlands,285446.340
EIRE,265545.900
Germany,228867.140
France,209024.050
Australia,138521.310
Spain,61577.110
Switzerland,56443.950
Belgium,41196.340


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [23]:
preferred_country = preferred_customer.groupby(['Country']).agg({'amount_spent':sum}).sort_values('amount_spent', ascending=False)

vip_and_preferred = pd.merge(left = vip_country,
                                 right = preferred_country,
                                 how = 'inner', 
                                 left_on = "Country", 
                                 right_on= "Country")

vip_and_preferred

,amount_spent_x,amount_spent_y
Country,,
United Kingdom,7308391.554,2204199.671
EIRE,265545.900,4341.210
Germany,228867.140,87709.480
France,209024.050,68311.580
Australia,138521.310,10056.290
Spain,61577.110,21173.500
Switzerland,56443.950,19066.130
Belgium,41196.340,31184.220
Sweden,38378.330,2484.980
